## Pytorchを用いたニューラルネットワークの実装

***

### ネットワークの定義

***

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,6,3)
        self.conv2 = nn.Conv2d(6,16,3)
        self.fc1 = nn.Linear(16*6*6,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1,self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self,x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
net = Net()
print(net)


Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


実装はforward関数の定義だけでよく、backward関数は自動的に定義される。
モデルの学習可能なパラメータはnet.parameters()によって取得できる。

In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

10
torch.Size([6, 1, 3, 3])


ランダムな32x32の入力を試す

In [3]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 0.0479, -0.0457,  0.0151, -0.0079, -0.0697,  0.0590, -0.1466, -0.0519,
         -0.1181, -0.0207]], grad_fn=<AddmmBackward0>)


テンソルに蓄積している勾配のバッファーを一度すべて０に初期化し、その後、乱数入力による勾配を誤差逆伝播させる。

In [4]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [6]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1) #変数targetの形をoutpuitに合わせる
criterion = nn.MSELoss()

loss = criterion(output,target)
print(loss)

tensor(0.3844, grad_fn=<MseLossBackward0>)
